In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [2]:
def build_cgan():
    
    # Generator network
    generator = models.Sequential()
    generator.add(layers.Dense(256, activation='relu', input_dim=101))  # 101 is the total dimension after concatenation
    generator.add(layers.Reshape((8, 8, 4)))
    
    generator.add(layers.Conv2DTranspose(316, (5, 5), strides=(1, 1), padding='same', activation='relu'))
    generator.add(layers.BatchNormalization())
    
    generator.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', activation='relu'))
    generator.add(layers.BatchNormalization())
    
    generator.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', activation='relu'))
    generator.add(layers.BatchNormalization())
    
    generator.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', activation='relu'))
    generator.add(layers.BatchNormalization())
    
    generator.add(layers.Conv2DTranspose(1, (5, 5), strides=(4, 4), padding='same', activation='tanh'))

    # Discriminator network
    discriminator = models.Sequential()
    discriminator.add(layers.Input(shape=(256, 256, 1)))
    
    discriminator.add(layers.Conv2D(64, (5, 5), strides=(1, 1), padding='same', activation='relu'))
    discriminator.add(layers.BatchNormalization())
    
    discriminator.add(layers.Conv2D(128, (5, 5), strides=(1, 1), padding='same', activation='relu'))
    discriminator.add(layers.BatchNormalization())

#     discriminator.add(layers.Flatten())
    discriminator.add(layers.Dense(1, activation='sigmoid'))

    return generator, discriminator


# Define user ID and noise input layers
user_id_input = layers.Input(shape=(1,), dtype=tf.int32)
noise_input = layers.Input(shape=(100,))

# Create cGAN model
generator, discriminator = build_cgan()

# Define loss functions and optimizers for generator and discriminator
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = Adam(1e-4)
discriminator_optimizer = Adam(1e-4)

# Define constants
BATCH_SIZE = 64
EPOCHS = 100
NUM_USERS = 10

# Training loop
@tf.function
def train_step(images, user_ids):
    noise = tf.random.normal([BATCH_SIZE, 100])
    user_id_float = layers.Lambda(lambda x: tf.cast(x, tf.float32))(user_ids)
    user_id_float = tf.reshape(user_id_float, (BATCH_SIZE, 1))
    # print(noise.shape, user_id_float.shape)
    concatenated_inputs = tf.concat([user_id_float, noise], axis=1)  # Concatenate user ID and noise
    # print(concatenated_inputs.shape)
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(concatenated_inputs, training=True)
        # print(generated_images)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [3]:
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               26112     
                                                                 
 reshape (Reshape)           (None, 8, 8, 4)           0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 316)        31916     
 nspose)                                                         
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 316)        1264      
 ormalization)                                                   
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 16, 16, 256)      2022656   
 ranspose)                                                       
                                                        

In [4]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1664      
                                                                 
 batch_normalization_4 (Batc  (None, 256, 256, 64)     256       
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 128)     204928    
                                                                 
 batch_normalization_5 (Batc  (None, 256, 256, 128)    512       
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 256, 256, 1)       129       
                                                                 
Total params: 207,489
Trainable params: 207,105
Non-tr

In [ ]:
augmented_data_folder = r'datasets\augmented_jabcode'

NUM_USERS = 10
NUM_IMAGES_PER_USER = 10

# Initialize empty lists to store user data and user IDs
user_data = []
user_ids = []

# Loop through the folders for each user
for user_id in range(1, NUM_USERS + 1):
    user_data_folder = os.path.join(augmented_data_folder, f'user_{user_id}')
    
    user_images = []  # To store images for the current user
    user_id_array = np.array([user_id] * NUM_IMAGES_PER_USER, dtype=np.int32)  # User IDs for the current user
    
    for image_id in range(NUM_IMAGES_PER_USER):
        image_filename = f'jabcode_user_{user_id}_augmented_{image_id}.png'
        image_path = os.path.join(user_data_folder, image_filename)
        
        # Load and preprocess the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale image
        image = cv2.resize(image, (256, 256))  # Resize if needed
        image = np.reshape(image, (256, 256, 1))
        # print(image.shape)
        user_images.append(image)
    
    user_data.append(user_images)
    user_ids.append(user_id_array)

# Now you have user_data, a list of lists containing user-specific images, and user_ids, a list of user IDs corresponding to each user's images.

# You can use these arrays to train your cGAN.

# Assuming you have already loaded and organized your data as user_data and user_ids

# Create a user ID to condition mapping
user_ids_mapping = {}
for i in range(NUM_USERS):
    user_ids_mapping[i] = np.array([i] * BATCH_SIZE, dtype=np.int32)

# Main training loop
for epoch in range(EPOCHS):
    for user_id in range(NUM_USERS):
        for batch_images in user_data[user_id]:
            train_step(batch_images, user_ids_mapping[user_id])